In [193]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from dataprep.eda import plot, plot_correlation, plot_missing

In [124]:
URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

In [125]:
URI_TREINO = "https://github.com/tgcsantos/quaretenadados/blob/master/DADOS_TREINO.csv?raw=true"
URI_TESTE = "https://github.com/tgcsantos/quaretenadados/raw/master/DADOS_TESTE.csv"
URI_DESAFIOQT = "https://github.com/tgcsantos/quaretenadados/raw/master/DESAFIOQT.csv"

dados_treino = pd.read_csv(URI_TREINO)
dados_teste = pd.read_csv(URI_TESTE)
dados_desafioqt = pd.read_csv(URI_DESAFIOQT)

erro_treino = "Erro ao carregar dados de treino"
erro_teste = "Erro ao carregar dados de teste"
erro_desafioqt = "Erro ao carregar dados de submissão"

assert dados_treino.shape == (150000, 5), erro_treino
assert dados_teste.shape == (20000, 5), erro_teste
assert dados_desafioqt.shape == (10000, 5), erro_desafioqt

Vemos que os dados estão divididos em dois datasets distintos, dados_treino e dados_teste. Pretendemos usar as técnicas de validação cruzada nos dados, por esse motivo não é necessário a divisão, verificaremos apenas se as distribuições nos dados são semelhantes, caso haja alguma diferença entre elas pode haver algum viés de seleção nas amostragens. Utilizaremos o  teste de Kolmogorov-Smirnov para isso.

In [126]:
from scipy.stats import ks_2samp
np.random.seed(42)
df_ks = pd.DataFrame(index =dados_treino.columns )
ks_list = []
pvalue_list = []
for element in dados_treino:
    ks_value = ks_2samp(dados_treino[element], dados_teste[element])
    ks_list.append(ks_value[0])
    pvalue_list.append(ks_value[1])
df_ks['ks_value'] = ks_list
df_ks['p_value'] = pvalue_list
df_ks.head()

,ks_value,p_value
NU_NOTA_CN,0.006917,0.367268
NU_NOTA_CH,0.005953,0.559092
NU_NOTA_LC,0.004760,0.818809
NU_NOTA_MT,0.005450,0.671017
NU_NOTA_REDACAO,0.003960,0.944800


Como podemos ver o valor do test KS é muito pequeno e o pvalue muito grande, sendo assim podemos concluir que as distribuições são parecidas, a amostragem deve ter sido feita de forma aleatória, portanto, para nossos fins uniremos as bases disponibilizadas em apenas uma.

In [127]:
dataset = pd.concat([dados_treino,dados_teste])
dataset = dataset.sample(frac=1, axis=0).reset_index(drop=True)
df_train_target = dataset['NU_NOTA_LC']
dataset.drop('NU_NOTA_LC',axis=1,inplace=True)

Verificando a distribuição

In [128]:
plot(dataset)

In [129]:
plot_missing(dataset)

In [130]:
plot_correlation(dataset)

In [131]:
df_train = dataset

In [132]:
df_train.columns

Index(['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], dtype='object')

In [133]:
df_train = data_transform(df_train)

In [134]:
X_train,X_test,y_train,y_test=train_test_split(df_train,df_train_target,test_size=0.30,random_state=0)

In [135]:
lm = LinearRegression()
lm.fit(X_train, y_train)
y_pred = lm.predict(X_test)


#Fiting Linear Regression (OLS)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error:35.756746783646
Mean Squared Error:2128.150575564355
Root Mean Squared Error:46.131882419475964


In [137]:
#Fiting Default RF regression (OLS)
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


Mean Absolute Error:37.460422549019604
Mean Squared Error:2344.0952842960783
Root Mean Squared Error:48.41585777713825


In [239]:
# Check hyperparametrts to RF Regression

from skopt import gp_minimize

resultados_gp = gp_minimize(treinar_modelo_rf, space, random_state=70, verbose=1, n_calls=40, n_random_starts=10,n_jobs=8)

Iteration No: 1 started. Evaluating function at random point.
[13.153092357169381, True]

Ridge regression
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0279
Function value obtained: 3728.1480
Current minimum: 3728.1480
Iteration No: 2 started. Evaluating function at random point.
[2.510563843876221, True]

Ridge regression
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0279
Function value obtained: 2545.8355
Current minimum: 2545.8355
Iteration No: 3 started. Evaluating function at random point.
[17.23240003194174, True]

Ridge regression
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0309
Function value obtained: 3955.9473
Current minimum: 2545.8355
Iteration No: 4 started. Evaluating function at random point.
[6.8321247810846275, True]

Ridge regression
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0269
Function value obtained: 3169.9407
Current minimum: 2545.8355
Iteration No: 5 started. E

In [240]:
resultados_gp.x

[0.004239261846890154, True]

In [168]:
# FITING RF WITH OPTIMAL HYPERPARAETERS

rf_optimal = RandomForestRegressor( max_depth = 9,n_estimators = 1842,bootstrap = True,max_features = 'auto',min_samples_leaf = 1,min_samples_split = 10)
rf_optimal.fit(X_train, y_train)
y_pred = rf_optimal.predict(X_test)


#Fiting RF Regression with optimal hyperparameters (OLS)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error:34.700046986095394
Mean Squared Error:2018.7537448236135
Root Mean Squared Error:44.93054356252118


In [257]:
# TRAINING RIDGE WITH OPTIMAL HYPERPARAETERS

resultados_gp = gp_minimize(treinar_modelo_lm, space,random_state=42, verbose=1, n_calls=60, n_random_starts=10,n_jobs=8)

Iteration No: 1 started. Evaluating function at random point.
[0.796563332561547, True]

Ridge regression
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0249
Function value obtained: 2250.7011
Current minimum: 2250.7011
Iteration No: 2 started. Evaluating function at random point.
[0.7797130311727423, False]

Ridge regression
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0269
Function value obtained: 2247.7724
Current minimum: 2247.7724
Iteration No: 3 started. Evaluating function at random point.
[0.4458881695783059, True]

Ridge regression
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0269
Function value obtained: 2189.4845
Current minimum: 2189.4845
Iteration No: 4 started. Evaluating function at random point.
[0.4593029670766707, True]

Ridge regression
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0279
Function value obtained: 2191.8283
Current minimum: 2189.4845
Iteration No: 5 started.

In [258]:
resultados_gp.x

[0.004277105745591676, True]

In [259]:
# FITING RIDGE WITH OPTIMAL HYPERPARAMETERS

ridge = Ridge(alpha=0.004277105745591676,normalize=True)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_test)


print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error:35.76301899423536
Mean Squared Error:2128.0934129210605
Root Mean Squared Error:46.13126285851126


In [252]:
resultados_gp = gp_minimize(treinar_modelo_lm,space,random_state=143, verbose=1, n_calls=50, n_random_starts=10,n_jobs=8)

Iteration No: 1 started. Evaluating function at random point.
[0.7734761561449972, False]

Lasso regression
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0359
Function value obtained: 5222.2180
Current minimum: 5222.2180
Iteration No: 2 started. Evaluating function at random point.
[0.8385118054146918, True]

Lasso regression
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0349
Function value obtained: 5222.2180
Current minimum: 5222.2180
Iteration No: 3 started. Evaluating function at random point.
[0.3394687464928691, False]

Lasso regression
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0359
Function value obtained: 5222.2180
Current minimum: 5222.2180
Iteration No: 4 started. Evaluating function at random point.
[0.4546328932701347, False]

Lasso regression
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0359
Function value obtained: 5222.2180
Current minimum: 5222.2180
Iteration No: 5 start

In [227]:
resultados_gp.x

[0.004017548610957156, False]

In [197]:
def treinar_modelo_rf(params):
    max_depth = params[0]
    n_estimators = params[1]
    bootstrap = params[2]
    max_features = params[3]
    min_samples_leaf = params[4]
    min_samples_split = params[5]
    
    print(params, '\n')
    
    mdl = RandomForestRegressor(random_state = 42, max_depth = max_depth, n_estimators=n_estimators)
    mdl.fit(X_train, y_train)
    
    p = mdl.predict(X_test)
    
    return metrics.mean_squared_error(y_test, p)


space = [(1, 10),
        (10, 2000),
        (True, False),
         ('auto', 'sqrt'),
         (1,4),
         (2,10) ]
          

In [255]:
def treinar_modelo_lm(params):
    alpha_ridge = params[0]
    normalize = params[1]
    type_regression = "Ridge"
    print(params, '\n')
    if(type_regression == 'Lasso'):
        print(type_regression+ " regression")
        mdl = Lasso(alpha=alpha_ridge,normalize=True)
        mdl.fit(X_train, y_train)
    else:
        print(type_regression+ " regression")
        mdl = Ridge(alpha=alpha_ridge,normalize=True)
        mdl.fit(X_train, y_train)
    
    p = mdl.predict(X_test)
    
    return metrics.mean_squared_error(y_test, p)

space = [(0.0001,1),
        (True, False)]

# resultado = dummy_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=30)

In [114]:
def data_transform(df):

    df['NU_EXATAS_MEDIA'] = (df['NU_NOTA_CN']+df['NU_NOTA_MT'])/2
    df['NU_HUMAN_MEDIA'] = (df['NU_NOTA_REDACAO']+df['NU_NOTA_CH'])/2
    df['NU_GERAL_MEDIA'] = (df['NU_NOTA_CN']+df['NU_NOTA_MT']+df['NU_NOTA_REDACAO']+df['NU_NOTA_CH'])/4

    df['CAT_EXATAS_MEDIA'] =  np.where(df['NU_EXATAS_MEDIA'].between(0, 456.000000, inclusive=True),'MUITO_BAIXO',
            np.where(df['NU_EXATAS_MEDIA'].between(456.000001, 499.450000, inclusive=True),'BAIXO',
            np.where(df['NU_EXATAS_MEDIA'].between(499.450001, 558.900000, inclusive=True),'MEDIO',
            np.where(df['NU_EXATAS_MEDIA'].between(558.900001, 700, inclusive=True),'ALTO','MUITO_ALTO'))))
    df['CAT_HUMAN_MEDIA'] = np.where(df['NU_HUMAN_MEDIA'].between(0, 458.400000, inclusive=True),'MUITO_BAIXO',
            np.where(df['NU_HUMAN_MEDIA'].between(458.400001, 542.750000, inclusive=True),'BAIXO',
            np.where(df['NU_HUMAN_MEDIA'].between(542.750000, 620.100000, inclusive=True),'MEDIO',
            np.where(df['NU_HUMAN_MEDIA'].between(620.100001, 700, inclusive=True),'ALTO','MUITO_ALTO'))))

    no_numerical_columns = df.select_dtypes(exclude='number')
    numerical_columns = df.select_dtypes(include='number')

    le = preprocessing.LabelEncoder()
    no_numerical_columns = no_numerical_columns.apply(le.fit_transform)
    df = pd.concat([no_numerical_columns, numerical_columns], axis=1)

    return df